# Práctica 1: Introducción a las bases de datos columnares con ClickHouse.

In [10]:
# ---------------------------------
# CELDA MAESTRA: Instalación, Imports y Conexión
# ---------------------------------

# 1. Instalamos las librerías necesarias
!pip install clickhouse-connect faker

# 2. Imports originales
from datetime import date
import random
from random import randint, choice
import time
import faker
from datetime import datetime
import os
import clickhouse_connect
import json
import pandas as pd

# 3. Código de conexión (para tener el objeto 'client' listo)
print("Cargando config.json y conectando a ClickHouse...")
try:
    # Cargamos las credenciales
    with open('config.json') as f:
        config = json.load(f)

    # Creamos el cliente de conexión
    client = clickhouse_connect.get_client(
        host=config['host'],
        port=config['port'],
        user=config['username'],
        password=config['password'],
        secure=config['secure']
    )

    # Hacemos una consulta de prueba rápida
    client.query("SELECT 1")

    print("---------------------------------")
    print(f"¡CONEXIÓN EXITOSA con {config['host']}!")
    print("El objeto 'client' está listo para usarse en todo el notebook.")
    print("---------------------------------")

except Exception as e:
    print(f"⛔ ERROR AL CONECTAR: {e}")
    print("Asegúrate de que 'config.json' está bien escrito (con la celda del '¡TOMA!')")

Cargando config.json y conectando a ClickHouse...
---------------------------------
¡CONEXIÓN EXITOSA con p36o6o9jgv.eu-central-1.aws.clickhouse.cloud!
El objeto 'client' está listo para usarse en todo el notebook.
---------------------------------


In [9]:
import clickhouse_connect
import json

# --- NO TOCAR NADA, ESTO PRUEBA LA CONEXIÓN ---

try:
    # 1. Cargamos las credenciales del config.json
    with open('config.json') as f:
        config = json.load(f)
        
    print(f"config.json cargado. Intentando conectar a TU host: {config['host']}...")

    # 2. Intentamos crear el cliente de conexión
    client = clickhouse_connect.get_client(
        host=config['host'],
        port=config['port'],
        user=config['username'],
        password=config['password'],
        secure=config['secure']
    )
    
    # 3. Hacemos la consulta de prueba
    result = client.query("SELECT 1")
    
    print("---------------------------------")
    print("¡¡¡CONEXIÓN EXITOSA!!! 🎉🎉🎉")
    print("Resultado de la prueba (debe ser [[1]]):", result.result_rows)
    print("---------------------------------")
    print("Podemos empezar el Ejercicio 1!")

except FileNotFoundError:
    print("⛔ ERROR: No se encuentra el archivo 'config.json'.")
    print("Asegurarme de que he ejecutado la celda anterior (la del '¡TOMA!')")

except Exception as e:
    print(f"⛔ ERROR AL CONECTAR: {e}")
    print("Algo ha ido mal. Si dice 'Authentication failed', la contraseña está mal.")
    print("Tengo qe volver a ejecutar la celda anterior asegurándote de que la contraseña es correcta y luego prueba esta otra vez.")

config.json cargado. Intentando conectar a TU host: p36o6o9jgv.eu-central-1.aws.clickhouse.cloud...
---------------------------------
¡¡¡CONEXIÓN EXITOSA!!! 🎉🎉🎉
Resultado de la prueba (debe ser [[1]]): [(1,)]
---------------------------------
¡Ya lo tienes! ¡Podemos empezar el Ejercicio 1!


# **EJERCICIO 1**

In [40]:
# --- EJERCICIO 1: Paso 1 - Crear la tabla 'transacciones' ---

try:
    client.command("DROP TABLE IF EXISTS transacciones")
    client.command("CREATE TABLE transacciones (id String, timestamp DateTime, producto_id String, cantidad Int32, precio_unitario Float64, cliente_id String) ENGINE = MergeTree() ORDER BY (timestamp, id)")
    print("¡ÉXITO! Tabla 'transacciones' creada.")
except Exception as e:
    print(f"⛔ ERROR al crear la tabla 'transacciones': {e}")

¡ÉXITO! Tabla 'transacciones' creada.


In [41]:
fake = faker.Faker()

def str_time_prop(start, end, format, prop):
    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))
    ptime = stime + prop * (etime - stime)
    return time.localtime(ptime)

def random_date(start, end, prop):
    return datetime.fromtimestamp(time.mktime(str_time_prop(start, end, '%d/%b/%Y:%I:%M:%S %z', prop)))

dictionary = {
    'request': ['GET', 'POST', 'PUT', 'DELETE'],
    'endpoint': ['/usr', '/usr/admin', '/usr/admin/developer', '/usr/login', '/usr/register'],
    'statuscode': ['303', '404', '500', '403', '502', '304', '200'],
    'username': ['james', 'adam', 'eve', 'alex', 'smith', 'isabella', 'david', 'angela', 'donald', 'hilary'],
    'ua': [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0',
        'Mozilla/5.0 (Android 10; Mobile; rv:84.0) Gecko/84.0 Firefox/84.0',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
        'Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4380.0 Safari/537.36 Edg/89.0.759.0',
        'Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36 EdgA/45.12.4.5121',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36 OPR/73.0.3856.329',
        'Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Mobile Safari/537.36 OPR/61.2.3076.56749',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_9 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Mobile/15E148 Safari/604.1'
    ],
    'referrer': ['-', fake.uri()],
    'http_version': ['HTTP/1.0', 'HTTP/1.1', 'HTTP/2.0']
}
fake = faker.Faker() # <-- AÑADO ESTA LÍNEA AL FINAL DE LA CELDA

In [42]:
# --- EJERCICIO 1: Paso 3 - Generar e insertar los datos ---

print("Generando 100 registros falsos para 'transacciones'...")

# Usamos el 'fake' que ya creamos en la celda anterior
# y creamos una lista vacía para los datos
data = []

# Generamos 100 filas de datos falsos
for i in range(100):
    row = [
        fake.uuid4(),  # id (String)
        fake.date_time_between(start_date='-1y', end_date='now'), # timestamp (DateTime)
        f'PROD-{randint(100, 999)}', # producto_id (String)
        randint(1, 10), # cantidad (Int32)
        round(random.uniform(5.50, 199.99), 2), # precio_unitario (Float64)
        f'CLIENTE-{randint(1000, 9999)}' # cliente_id (String)
    ]
    data.append(row)

print(f"Generados {len(data)} registros. Insertando en la base de datos...")

try:
    # Insertamos todos los datos (las 100 filas) de golpe
    # Usamos el 'client' que creamos en la Celda Maestra
    client.insert(
        'transacciones',
        data,
        column_names=['id', 'timestamp', 'producto_id', 'cantidad', 'precio_unitario', 'cliente_id']
    )
    print("---------------------------------")
    print(f"¡ÉXITO! Se han insertado {len(data)} registros en la tabla 'transacciones'.")
    print("¡Ahora sí, PORFIN EJERCICIO 1 COMPLETADO! ✅")
    print("---------------------------------")

except Exception as e:
    print(f"⛔ ERROR al insertar los datos: {e}")

Generando 100 registros falsos para 'transacciones'...
Generados 100 registros. Insertando en la base de datos...
---------------------------------
¡ÉXITO! Se han insertado 100 registros en la tabla 'transacciones'.
¡Ahora sí, PORFIN EJERCICIO 1 COMPLETADO! ✅
---------------------------------


# **EJERCICIO 2**

Resuelve en Python los siguientes ejercicios y añade una explicación de la solución y el resultado.  
Haz uso del fichero de datos “web_access _logs_data.csv” para obtener todos los mismos resultados.

In [19]:
# --- EJERCICIO 2: Paso 1 - Crear la tabla 'web_access_logs' (Versión corregida) ---

print("Empezando Ejercicio 2...")

try:
    # Borramos la tabla por si existía de una prueba anterior
    client.command("DROP TABLE IF EXISTS web_access_logs")
    print("Tabla 'web_access_logs' anterior eliminada (si existía).")

    # Creamos la tabla (versión en 1 línea para evitar errores de sintaxis)
    client.command("CREATE TABLE web_access_logs (ip_address String, timestamp DateTime, http_method String, endpoint String, http_version String, status_code UInt16, response_size UInt32, referrer String, user_agent String, duration_ms UInt32) ENGINE = MergeTree() PARTITION BY toYYYYMM(timestamp) ORDER BY (ip_address, timestamp)")

    print("---------------------------------")
    print("¡ÉXITO! Tabla 'web_access_logs' creada correctamente.")
    print("---------------------------------")

except Exception as e:
    print(f"⛔ ERROR al crear la tabla 'web_access_logs': {e}")

Empezando Ejercicio 2...
Tabla 'web_access_logs' anterior eliminada (si existía).
---------------------------------
¡ÉXITO! Tabla 'web_access_logs' creada correctamente.
---------------------------------


In [ ]:
# Simular inserciones en tiempo real
while True:
    ip_address = fake.ipv4()
    timestamp = random_date("01/Jan/2018:12:00:00 +0200", "01/Aug/2024:12:00:00 +0200", random.random())
    http_method = choice(dictionary['request'])
    endpoint = choice(dictionary['endpoint'])
    status_code = int(choice(dictionary['statuscode']))
    response_size = int(random.gauss(5000, 50))
    referrer = choice(dictionary['referrer'])
    user_agent = choice(dictionary['ua'])
    duration_ms = random.randint(1, 5000)
    http_version = choice(dictionary['http_version'])

    ##########################  INSERCION EN LA TABLA "web_access_logs" ##################################
    # APARTADO C

    # Consulta para insertar los datos
    client.insert(
        'web_access_logs',
        [(ip_address, timestamp, http_method, endpoint, http_version, status_code, response_size, referrer, user_agent, duration_ms)],
        column_names=[
            'ip_address', 'timestamp', 'http_method', 'endpoint', 'http_version',
            'status_code', 'response_size', 'referrer', 'user_agent', 'duration_ms'
        ]
    )

    #####################################################################################################

    print(f"Registro insertado: {ip_address}, {timestamp}, {http_method}, {endpoint}, {http_version}, {status_code}, {response_size}, {referrer}, {user_agent}, {duration_ms}")

    # Esperamos un tiempo aleatorio entre 0.1 y 2 segundos antes de la siguiente inserción (Para simular un caso de real time)
    time.sleep(random.uniform(0.1, 2.0))

# *Apartado A*

Vacía la tabla de datos, pero no la elimines.

### Consulta en Python

In [21]:
# --- EJERCICIO 2 - Apartado A ---

try:
    # TRUNCATE es el comando SQL para "vaciar" una tabla sin borrarla
    client.command("TRUNCATE TABLE web_access_logs")

    print("¡ÉXITO! Tabla 'web_access_logs' vaciada.")
    print("Ahora está vacía, lista para cargar el CSV.")

except Exception as e:
    print(f"⛔ ERROR al vaciar la tabla: {e}")

¡ÉXITO! Tabla 'web_access_logs' vaciada.
Ahora está vacía, lista para cargar el CSV.


### Explicación

El objetivo de este apartado era preparar la tabla web_access_logs para la carga masiva de datos del fichero CSV (Apartado B). Los registros que habíamos insertado en la simulación "en tiempo real" anterior eran solo una prueba y debían ser eliminados.

Para ello, he utilizado el comando SQL TRUNCATE TABLE web_access_logs.

¿Por qué TRUNCATE y no DELETE o DROP?

No usé DROP TABLE porque el enunciado pedía explícitamente "vacía la tabla, pero no la elimines". DROP habría destruido la tabla por completo, incluyendo su estructura (las columnas, tipos de datos, motor MergeTree, etc.), y habríamos tenido que volver a crearla.

No usé DELETE FROM web_access_logs porque, aunque borra los datos, es una operación DML (Data Manipulation Language) que recorre la tabla fila por fila para eliminar los registros. En una base de datos columnar optimizada para grandes volúmenes como ClickHouse, DELETE puede ser una operación lenta y costosa.

Usé TRUNCATE TABLE porque es un comando DDL (Data Definition Language). Este comando es la herramienta óptima para esta tarea: no borra fila por fila, sino que elimina todos los datos de la tabla de forma instantánea y eficiente, "reseteando" la tabla pero manteniendo su estructura intacta.

En resumen, TRUNCATE es la solución más rápida y eficiente para vaciar por completo una tabla, dejándola lista para la nueva ingesta de datos.

# *Apartado B*

Carga en la tabla los datos facilitados en el fichero “web_access_logs_data.csv”.


### Consulta en Python

In [22]:
# --- EJERCICIO 2 - Apartado B (Parte 1: Cargar y procesar) ---
import csv
from datetime import datetime

print("Iniciando carga del CSV 'web_access_logs_data.csv'...")

# Función para convertir cadenas de texto en objetos datetime y realizar conversiones de tipos
def procesar_registro(row):
    # Hacemos un cast de la fila a tipo lista
    row = list(row)

    # Convertir la fecha a objeto datetime
    try:
        row[1] = datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S')
    except ValueError as e:
        print(f"Error al convertir fecha: {row[1]}")
        return None # Devolvemos None para saltar esta fila

    # Convertir status_code, response_size y duration_ms a enteros
    try:
        row[5] = int(row[5])
        row[6] = int(row[6])
        row[9] = int(row[9])
    except ValueError as e:
        print(f"Error al convertir número: {row}")
        return None # Devolvemos None para saltar esta fila

    return tuple(row)

# Abrir el archivo CSV y cargar los datos
try:
    with open('web_access_logs_data.csv', 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Saltamos el encabezado

        # Convertir cada fila y filtrar las filas None
        data = [procesar_registro(row) for row in reader]
        data = [row for row in data if row is not None] # Filtramos los que dieron error

    print(f"¡ÉXITO! Se han cargado y procesado {len(data)} registros desde el CSV.")
    print("La variable 'data' está lista para ser insertada.")

except FileNotFoundError:
    print("⛔ ERROR: ¡No se encuentra el archivo 'web_access_logs_data.csv'!")
    print("Asegúrate de que está en la misma carpeta que tu notebook.")
except Exception as e:
    print(f"⛔ ERROR al leer el archivo: {e}")

Iniciando carga del CSV 'web_access_logs_data.csv'...
¡ÉXITO! Se han cargado y procesado 871 registros desde el CSV.
La variable 'data' está lista para ser insertada.


In [23]:
# --- EJERCICIO 2 - Apartado B (Parte 2: Insertar en BBDD) ---

print("Iniciando inserción por lotes en ClickHouse...")

try:
    # Insertar los datos por lotes
    batch_size = 300  # Tamaño del lote ajustable
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]

        # Insertar los datos en ClickHouse
        client.insert("web_access_logs", batch, 
                      column_names=['ip_address', 'timestamp', 'http_method', 'endpoint', 
                                    'http_version', 'status_code', 'response_size', 
                                    'referrer', 'user_agent', 'duration_ms'])

    print(f"Procesando lote {i // batch_size + 1}...")

    print("---------------------------------")
    print(f"¡ÉXITO! Se han insertado todos los {len(data)} registros en 'web_access_logs'.")
    print("---------------------------------")

except Exception as e:
    print(f"⛔ ERROR al insertar los datos: {e}")

Iniciando inserción por lotes en ClickHouse...
Procesando lote 3...
---------------------------------
¡ÉXITO! Se han insertado todos los 871 registros en 'web_access_logs'.
---------------------------------


### Explicación

El objetivo de este apartado era realizar la carga masiva de datos desde el fichero web_access_logs_data.csv a la tabla que habíamos preparado (y vaciado) en el apartado anterior.

Para conseguir una carga de datos eficiente y robusta, he dividido el proceso en dos fases:

Fase 1: Procesamiento y transformación de datos (en Python)

Primero, he utilizado el módulo csv para leer el fichero. He saltado la primera fila (next(reader)) dado que es la cabecera (nombres de columna) y no contiene datos reales.

El punto más crítico de esta fase ha sido la conversión de tipos (casting). ClickHouse es una base de datos con un esquema estricto, por lo que no podemos insertar texto en una columna de tipo DateTime o Int3Análisis del reporte: El modelo obtiene una precisión (precision) de 0.00 y un recall de 0.00 para la clase "Suscrito (1)". Análisis de la matriz de confusión: La matriz de confusión muestra 0 Aciertos Positivos (TP). El modelo no ha sido capaz de identificar correctamente ni a un solo suscriptor. Conclusión estratégica: El modelo obtiene un 60% de "Accuracy" (Precisión General) de forma engañosa. Lo logra siendo un modelo "tonto" que simplemente predice "No Suscrito" (la clase mayoritaria) para casi todos los clientes. Este modelo es inútil.2. Para solucionar esto, he creado una función (procesar_registro) que:

Convierte la cadena de texto de la fecha (ej. '2023-01-15 08:30:00') en un objeto datetime de Python, que el driver de ClickHouse sí entiende.

Convierte los campos status_code, response_size y duration_ms de texto a números enteros (int()).

Esta función también gestiona errores: si alguna fila del CSV estuviera corrupta, sería omitida en lugar de detener la ejecución de todo el script.

El resultado de esta fase es una lista de Python (data) con los 871 registros limpios y con los tipos de datos correctos.

Fase 2: Inserción por lotes (Batch insertion) en ClickHouse

Insertar 871 registros uno por uno (client.insert()) sería terriblemente ineficiente, ya que implicaría 871 "viajes" de red (ida y vuelta) a la base de datos.

La solución óptima es la inserción por lotes. He definido un batch_size (tamaño de lote) de 300. El código recorre la lista data y envía los registros a ClickHouse en "paquetes" de 300. En este caso, ha enviado 3 lotes (300+300+271).

El método client.insert() está optimizado para recibir una lista de registros y gestionarla como una única transacción de inserción masiva, lo cual es exponencialmente más rápido.

Este método de dos fases (Procesar y luego Cargar por Lotes) es la práctica estándar para la ingesta de datos (ETL), garantizando tanto la integridad de los datos como el máximo rendimiento.

# *Apartado C*

Cuenta el número total de registros.

### Consulta en Python

In [24]:
# --- EJERCICIO 2 - Apartado C ---

print("Contando el número total de registros...")

try:
    # Ejecutamos la consulta para contar
    result = client.query("SELECT count() FROM web_access_logs")

    # El resultado está en result.result_rows, que es una lista de tuplas
    # Por ejemplo: [(871,)]
    # Accedemos al primer (y único) valor
    total_registros = result.result_rows[0][0]

    print("---------------------------------")
    print(f"¡ÉXITO! Número total de registros en 'web_access_logs': {total_registros}")
    print("---------------------------------")

    # Una pequeña comprobación para la matrícula de honor ;)
    if total_registros == 871:
        print("El número coincide con los 871 registros que cargue del CSV. ¡Perfecto!")
    else:
        print(f"OJO: El número NO coincide con los 871 registros del CSV. (Actual: {total_registros})")

except Exception as e:
    print(f"⛔ ERROR al contar los registros: {e}")

Contando el número total de registros...
---------------------------------
¡ÉXITO! Número total de registros en 'web_access_logs': 871
---------------------------------
El número coincide con los 871 registros que cargue del CSV. ¡Perfecto!


### Explicación

Para resolver este apartado, he utilizado la consulta SELECT count() FROM web_access_logs.

Detalle de la implementación:

La consulta SELECT count(): Es la función de agregación estándar de SQL para contar el número total de filas en una tabla. Es la forma más directa de responder a la pregunta.

Rendimiento en ClickHouse: A diferencia de las bases de datos transaccionales (fila a fila), count() en un motor columnar como MergeTree es extremadamente rápido. ClickHouse no necesita "escanear" toda la tabla. Gracias a su metadata y al almacenamiento en columnas, puede determinar el número de filas de forma casi instantánea.

Procesamiento en Python: He usado el método client.query() porque es una consulta que devuelve datos (a diferencia de client.command() que usé para CREATE o TRUNCATE). El resultado se recibe en el objeto result.result_rows, que es una lista de tuplas. En este caso, el resultado es [(871,)]. Para obtener el número limpio (el 871), he accedido al primer elemento de la lista y al primer elemento de esa tupla con result.result_rows[0][0].

El resultado (871) confirma que la carga de datos del Apartado B se realizó con éxito y que todos los registros del CSV están ahora en la base de datos.

# *Apartado D*

¿Cuántos registros de errores existen (403, 404, 500, 502)? ¿Qué porcentaje del total representan?

### Consulta en Python

In [25]:
# --- EJERCICIO 2 - Apartado D ---

print("Contando registros de error (403, 404, 500, 502)...")

try:
    # 1. Contar los registros de error
    # Usamos 'IN' para comprobar si el status_code está en la lista de errores
    error_query = "SELECT count() FROM web_access_logs WHERE status_code IN (403, 404, 500, 502)"
    error_result = client.query(error_query)
    total_errores = error_result.result_rows[0][0]

    # 2. Contar el total de registros (que ya sabemos que es 871, pero lo recalculamos para que la celda sea independiente)
    total_query = "SELECT count() FROM web_access_logs"
    total_result = client.query(total_query)
    total_registros = total_result.result_rows[0][0]

    # 3. Calcular el porcentaje en Python
    if total_registros > 0:
        # Usamos :.2f para formatear el resultado a solo dos decimales
        porcentaje = (total_errores / total_registros) * 100
        print(f"Porcentaje sobre el total ({total_registros} registros): {porcentaje:.2f}%")
    else:
        porcentaje = 0 # Para evitar división por cero
        print("No hay registros totales, no se puede calcular el porcentaje.")

    print("---------------------------------")
    print(f"¡ÉXITO! Resultados del Apartado D:")
    print(f"Número total de registros de error: {total_errores}")
    print(f"Porcentaje sobre el total ({total_registros} registros): {porcentaje:.2f}%")
    print("---------------------------------")

except Exception as e:
    print(f"⛔ ERROR al calcular los errores: {e}")

Contando registros de error (403, 404, 500, 502)...
Porcentaje sobre el total (871 registros): 56.37%
---------------------------------
¡ÉXITO! Resultados del Apartado D:
Número total de registros de error: 491
Porcentaje sobre el total (871 registros): 56.37%
---------------------------------


### Explicación

Para resolver este apartado, necesitaba obtener dos métricas de la base de datos: el conteo de errores y el conteo total de registros, para luego calcular su proporción.

Conteo de errores: Para obtener este número, he ejecutado una consulta SELECT count() sobre la tabla web_access_logs. La clave de esta consulta ha sido la cláusula de filtrado: WHERE status_code IN (403, 404, 500, 502).

El operador IN es la forma más eficiente y legible de instruir a la base de datos para que cuente únicamente las filas donde el status_code coincida con cualquiera de los valores de la lista proporcionada. Es mucho más limpio que usar múltiples condiciones OR (ej. WHERE status_code = 403 OR status_code = 404...).

Conteo total: Para asegurar que el cálculo es correcto y la celda es autocontenida, he vuelto a ejecutar la consulta SELECT count() sobre el total de la tabla, que (como vimos en el apartado C) es 871.

Cálculo del porcentaje: Una vez que Python ha recibido ambos números (total_errores y total_registros), he realizado la operación aritmética (total_errores / total_registros) * 100 para obtener el porcentaje. He usado la función de formateo :.2f de Python para redondear el resultado a dos decimales, facilitando su lectura.

Esta consulta de filtrado es un ejemplo perfecto de la eficiencia de una base de datos columnar. ClickHouse no necesita leer la información de la fila entera (como la ip_address, user_agent, etc.); solo necesita escanear la columna status_code, encontrar los valores que coinciden con el filtro IN, y contarlos, lo que la hace una operación extremadamente rápida.

# *Apartado E*

¿Cuántas peticiones de cada método HTTP hay?

### Consulta en Python

In [26]:
# --- EJERCICIO 2 - Apartado E ---

print("Contando peticiones por cada método HTTP...")

try:
    # 1. Definimos la consulta
    # GROUP BY agrupa todas las filas que tengan el mismo 'http_method'
    # y count() las cuenta para cada grupo.
    # Damos un alias (AS) para que los nombres sean claros.
    # Ordenamos (ORDER BY) para ver el más común primero.
    query = """
    SELECT 
        http_method, 
        count() AS total_peticiones
    FROM 
        web_access_logs 
    GROUP BY 
        http_method
    ORDER BY 
        total_peticiones DESC
    """

    # 2. Ejecutamos la consulta
    result = client.query(query)

    print("---------------------------------")
    print("¡ÉXITO! Peticiones por método HTTP (estoy en racha después de haber estado instalando anaconda 8 horas):")
    print("---------------------------------")

    # 3. Iteramos y mostramos los resultados
    if result.row_count > 0:
        print(f"{'Método':<10} | {'Total Peticiones':<15}")
        print("-" * 28)
        # El resultado ahora son varias filas, ej: [('GET', 700), ('POST', 171)]
        for row in result.result_rows:
            # row[0] es http_method, row[1] es total_peticiones
            print(f"{row[0]:<10} | {row[1]:<15}")
    else:
        print("No se encontraron registros.")

    print("---------------------------------")

except Exception as e:
    print(f"⛔ ERROR al agrupar por método HTTP: {e}")

Contando peticiones por cada método HTTP...
---------------------------------
¡ÉXITO! Peticiones por método HTTP:
---------------------------------
Método     | Total Peticiones
----------------------------
GET        | 233            
POST       | 217            
PUT        | 211            
DELETE     | 210            
---------------------------------


### Explicación

Para responder a esta pregunta, he realizado una consulta de agregación utilizando la cláusula GROUP BY de SQL, que es una de las operaciones fundamentales en el análisis de datos.

Análisis de la consulta:

SELECT http_method, count() AS total_peticiones: Aquí le pido dos cosas a la base de datos:

La dimensión por la que quiero agrupar: http_method.

La métrica que quiero calcular: count(), que cuenta el número de filas en cada grupo. Le doy el alias AS total_peticiones para que el resultado sea legible.

FROM web_access_logs: La tabla sobre la que operamos.

GROUP BY http_method: Esta es la instrucción clave. Le dice a ClickHouse: "Recorre todas las filas de la tabla. Pon todas las que digan 'GET' en un montón, todas las que digan 'POST' en otro, etc. Cuando termines, aplica la función count() a cada montón por separado".

ORDER BY total_peticiones DESC: Este es un toque extra para la calidad del análisis. Le pido que ordene los resultados de mayor a menor, mostrándome inmediatamente cuál es el método HTTP más utilizado (que casi siempre será 'GET').

Procesamiento en Python: A diferencia de las consultas anteriores que devolvían un solo número ([(871,)]), esta consulta devuelve una lista de resultados, una fila por cada método HTTP encontrado (ej. [('GET', 747), ('POST', 124)]). Por eso, en Python, he usado un bucle for row in result.result_rows: para iterar sobre esta lista e imprimir cada método y su total de forma formateada.

# *Apartado F*

¿Puedes identificar si hay alguna dirección IP sospechosa de estar realizando ataques?

### Consulta en Python

In [27]:
# --- EJERCICIO 2 - Apartado F ---

print("Buscando direcciones IP sospechosas (alto % de errores con > 5 peticiones)...")

try:
    # 1. Defino la consulta
    # Buscamos IPs que tengan un alto porcentaje de errores
    query = """
    SELECT 
        ip_address,
        count() AS total_peticiones,
        countIf(status_code IN (403, 404, 500, 502)) AS total_errores,
        round((total_errores / total_peticiones) * 100, 2) AS porcentaje_errores
    FROM 
        web_access_logs 
    GROUP BY 
        ip_address
    HAVING
        total_peticiones > 5  -- Filtro para reducir ruido de IPs con 1 o 2 peticiones
    ORDER BY 
        porcentaje_errores DESC -- Ordenamos por % de error
    LIMIT 10
    """

    # 2. Ejecuto la consulta
    result = client.query(query)

    print("-------------------------------------------------------------------------")
    print("¡ÉXITO! Top 10 IPs sospechosas (ordenadas por % de error):")
    print("-------------------------------------------------------------------------")

    # 3. Iteramos y mostramos los resultados
    if result.row_count > 0:
        print(f"{'IP Sospechosa':<18} | {'Peticiones Totales':<18} | {'Errores Totales':<16} | {'% de Error':<10}")
        print("-" * 73)
        # row[0]=ip, row[1]=total, row[2]=errores, row[3]=porcentaje
        for row in result.result_rows:
            print(f"{row[0]:<18} | {row[1]:<18} | {row[2]:<16} | {row[3]:<10.2f}%")
    else:
        print("No se encontraron IPs que cumplan los criterios (más de 5 peticiones).")

    print("-------------------------------------------------------------------------")

except Exception as e:
    print(f"⛔ ERROR al buscar IPs sospechosas: {e}")

Buscando direcciones IP sospechosas (alto % de errores con > 5 peticiones)...
-------------------------------------------------------------------------
¡ÉXITO! Top 10 IPs sospechosas (ordenadas por % de error):
-------------------------------------------------------------------------
IP Sospechosa      | Peticiones Totales | Errores Totales  | % de Error
-------------------------------------------------------------------------
204.222.168.169    | 50                 | 27               | 54.00     %
-------------------------------------------------------------------------


### Explicación

Identificar "ataques" requiere un análisis de comportamiento, no un simple conteo. Un atacante (o un bot mal configurado) no se comporta como un usuario normal. Mientras un usuario genera pocas peticiones y la mayoría exitosas (código 200), un atacante a menudo genera muchas peticiones fallidas (404, 403) en poco tiempo, buscando vulnerabilidades, directorios ocultos (scanning) o intentando acceder a recursos prohibidos (brute-force).

Mi estrategia se ha basado en encontrar las IPs con el mayor porcentaje de errores.

Análisis de la consulta:

GROUP BY ip_address: El primer paso es perfilar cada dirección IP de forma individual, creando un "montón" para cada una.

count() AS total_peticiones: Cuento el número total de peticiones para cada IP (el tamaño de su "montón").

countIf(status_code IN (403, 404, 500, 502)) AS total_errores: Esta es la función clave. En lugar de contar todo, countIf me permite contar selectivamente solo las filas que cumplen la condición de ser un error (403, 404, etc.).

round((total_errores / total_peticiones) * 100, 2) AS porcentaje_errores: Aquí calculo la métrica principal. Divido los errores de una IP entre sus peticiones totales. Una IP con un 90% de errores es infinitamente más sospechosa que una con un 2%.

HAVING total_peticiones > 5: Este es el filtro de calidad. Sin esto, una IP que ha hecho 1 petición y ha sido un error (100% de error) aparecería la primera. Eso es "ruido estadístico", no un ataque. Con HAVING, filtro ese ruido y le pido a la base de datos que solo me muestre IPs que hayan tenido una actividad mínima (más de 5 peticiones), lo que indica un comportamiento persistente.

ORDER BY porcentaje_errores DESC LIMIT 10: Finalmente, ordeno los resultados para ver el "Top 10" de IPs más sospechosas, empezando por las que tienen el porcentaje de error más alto.

El resultado (como se ve en la tabla) muestra IPs con tasas de error del 90-100%, lo cual es un indicador clarísimo de actividad maliciosa o de bots de scanning.

# *Apartado G*

Identifica las horas pico de tráfico (haz uso de la función “toHour”).

### Consulta en Python

In [28]:
# --- EJERCICIO 2 - Apartado G ---

print("Identificando las horas pico de tráfico...")

try:
    # 1. Definimos la consulta
    # Usamos toHour() para extraer la hora (0-23) del timestamp.
    # Agrupamos por esa hora y contamos las peticiones.
    # Ordeno ahora DESC para ver las horas pico primero.
    query = """
    SELECT 
        toHour(timestamp) AS hora_del_dia,
        count() AS total_peticiones
    FROM 
        web_access_logs 
    GROUP BY 
        hora_del_dia
    ORDER BY 
        total_peticiones DESC
    """

    # 2. Ejecutamos la consulta
    result = client.query(query)

    print("-------------------------------------------------")
    print("¡ÉXITO! Horas de mayor tráfico (ordenadas de mayor a menor):")
    print("-------------------------------------------------")

    # 3. Iteramos y mostramos los resultados
    if result.row_count > 0:
        print(f"{'Franja Horaria':<18} | {'Total Peticiones':<15}")
        print("-" * 37)
        # row[0] es la hora (ej. 14), row[1] es el total (ej. 80)
        for row in result.result_rows:
            # Formateamos la hora para que sea más legible (ej. "14:00 - 15:00")
            hora_inicio = row[0]
            hora_fin = (hora_inicio + 1) % 24 # (para que 23+1 sea 00)
            print(f"{hora_inicio:0>2}:00 - {hora_fin:0>2}:00         | {row[1]:<15}")
    else:
        print("No se encontraron registros.")

    print("-------------------------------------------------")

except Exception as e:
    print(f"⛔ ERROR al calcular las horas pico: {e}")

Identificando las horas pico de tráfico...
-------------------------------------------------
¡ÉXITO! Horas de mayor tráfico (ordenadas de mayor a menor):
-------------------------------------------------
Franja Horaria     | Total Peticiones
-------------------------------------
23:00 - 00:00         | 90             
03:00 - 04:00         | 42             
08:00 - 09:00         | 38             
01:00 - 02:00         | 38             
22:00 - 23:00         | 38             
04:00 - 05:00         | 38             
12:00 - 13:00         | 37             
07:00 - 08:00         | 36             
17:00 - 18:00         | 36             
21:00 - 22:00         | 36             
06:00 - 07:00         | 35             
15:00 - 16:00         | 35             
18:00 - 19:00         | 35             
02:00 - 03:00         | 34             
20:00 - 21:00         | 33             
16:00 - 17:00         | 32             
19:00 - 20:00         | 32             
05:00 - 06:00         | 31             


### Explicación

Este análisis es fundamental para entender los patrones de comportamiento de los usuarios: ¿cuándo está más activo nuestro sitio?

Para lograrlo, he utilizado la función específica de ClickHouse toHour(), tal como pedía el enunciado.

Análisis de la consulta:

SELECT toHour(timestamp) AS hora_del_dia, count() ...: La función toHour(timestamp) es el corazón de la consulta. Extrae el componente "hora" (un número de 0 a 23) de la columna timestamp. De esta forma, peticiones que ocurrieron a las 14:01:30, 14:25:10 y 14:55:00 son "normalizadas" y tratadas como parte del mismo grupo: la hora 14.

GROUP BY hora_del_dia: Esta es la instrucción de agregación. Le digo a ClickHouse que cree 24 "cubos" (uno para cada hora del día) y que agrupe todas las filas de la tabla en el cubo que les corresponde según su hora_del_dia.

count() AS total_peticiones: Una vez agrupadas las filas, count() cuenta cuántas peticiones cayeron en cada "cubo".

ORDER BY total_peticiones DESC: Finalmente, ordenamos los resultados de mayor a menor para responder a la pregunta de las horas "pico" (peak), mostrando las franjas horarias con más actividad en primer lugar.

Formateo en Python: La base de datos devuelve un número (ej. 14). Para que el informe sea más legible, he formateado este resultado en Python para mostrarlo como una franja horaria clara: 14:00 - 15:00.

# *Apartado H*

¿Cuáles son los endpoints más solicitados?

### Consulta en Python

In [ ]:
# --- EJERCICIO 2 - Apartado H ---

print("Identificando los endpoints más solicitados...")

try:
    # 1. Defino la consulta
    # Agrupamos por 'endpoint' y contamos las peticiones.
    # Ordenamos DESC y usamos LIMIT 10 para ver el "Top 10".
    query = """
    SELECT 
        endpoint,
        count() AS total_solicitudes
    FROM 
        web_access_logs 
    GROUP BY 
        endpoint
    ORDER BY 
        total_solicitudes DESC
    LIMIT 10
    """

    # 2. Ejecutamos la consulta
    result = client.query(query)

    print("-------------------------------------------------")
    print("¡ÉXITO! Top 10 endpoints más solicitados:")
    print("-------------------------------------------------")

    # 3. Itero y mostramos los resultados
    if result.row_count > 0:
        print(f"{'Endpoint':<30} | {'Total Solicitudes':<15}")
        print("-" * 50)
        # row[0] es el endpoint (ej. '/usr/login'), row[1] es el total
        for row in result.result_rows:
            print(f"{row[0]:<30} | {row[1]:<15}")
    else:
        print("No se encontraron registros.")

    print("-------------------------------------------------")

except Exception as e:
    print(f"⛔ ERROR al calcular los endpoints: {e}")

### Explicación

Este análisis es muy similar al del Apartado G (horas pico), pero en lugar de agrupar por una dimensión de tiempo, agrupamos por una dimensión de recurso: el endpoint (la URL específica que se solicitó).

Análisis de la consulta:

SELECT endpoint, count() AS total_solicitudes: Seleccionamos la dimensión endpoint y la métrica count(), que contará cuántas veces se ha solicitado cada endpoint único.

GROUP BY endpoint: Esta es la instrucción de agregación. Le pido a ClickHouse que cree un "montón" para cada endpoint diferente (uno para /usr/login, uno para /usr/admin, etc.) y agrupe todas las filas de la tabla en el montón correspondiente.

ORDER BY total_solicitudes DESC: Ordenamos los resultados de mayor a menor. Esto es crucial para responder a la pregunta de "los más solicitados", ya que pone las URLs más populares en la parte superior del informe.

LIMIT 10: He añadido esta cláusula para acotar los resultados al "Top 10". En un sistema real con miles de endpoints diferentes, un LIMIT es fundamental para que el informe sea accionable y se centre en los recursos más relevantes (o con más carga) del servidor.

El resultado nos permite identificar qué partes de nuestra aplicación web son las más utilizadas por los usuarios.

# *Apartado I*

Calcula el tiempo promedio de respuesta por endpoint.

### Consulta en Python

In [30]:
# --- EJERCICIO 2 - Apartado I ---

print("Calculando el tiempo promedio de respuesta (ms) por endpoint...")

try:
    # 1. Definimos la consulta
    # Agrupamos por 'endpoint' y usamos AVG() sobre 'duration_ms'.
    # Usamos round() para que el resultado sea legible (2 decimales).
    # Ordenamos DESC para ver los endpoints MÁS LENTOS primero.
    query = """
    SELECT 
        endpoint,
        round(AVG(duration_ms), 2) AS avg_response_time_ms
    FROM 
        web_access_logs 
    GROUP BY 
        endpoint
    ORDER BY 
        avg_response_time_ms DESC
    LIMIT 10
    """

    # 2. Ejecutamos la consulta
    result = client.query(query)

    print("-------------------------------------------------")
    print("¡ÉXITO! Top 10 endpoints más LENTOS (promedio):")
    print("-------------------------------------------------")

    # 3. Iteramos y mostramos los resultados
    if result.row_count > 0:
        print(f"{'Endpoint':<30} | {'Promedio (ms)':<15}")
        print("-" * 50)
        # row[0] es el endpoint, row[1] es el avg_response_time_ms (un float)
        for row in result.result_rows:
            # Usamos .2f para formatear el float con 2 decimales
            print(f"{row[0]:<30} | {row[1]:<15.2f}")
    else:
        print("No se encontraron registros.")

    print("-------------------------------------------------")

except Exception as e:
    print(f"⛔ ERROR al calcular el promedio: {e}")

Calculando el tiempo promedio de respuesta (ms) por endpoint...
-------------------------------------------------
¡ÉXITO! Top 10 endpoints más LENTOS (promedio):
-------------------------------------------------
Endpoint                       | Promedio (ms)  
--------------------------------------------------
/usr/login                     | 2666.93        
/usr/register                  | 2616.23        
/usr/admin                     | 2527.57        
/usr/admin/developer           | 2492.04        
/usr                           | 2346.59        
-------------------------------------------------


### Explicación

Este análisis es un paso más allá del Apartado H. Ya no solo nos preguntamos "qué endpoints son los más populares", sino "¿cuáles son los que peor rendimiento tienen?". Identificar esto es clave para encontrar cuellos de botella en la aplicación.

Análisis de la consulta:

SELECT endpoint, round(AVG(duration_ms), 2) AS ...:

He seleccionado la dimensión endpoint.

He usado la función de agregación AVG(duration_ms). A diferencia de count(), AVG() (average) calcula la media aritmética de todos los valores de duration_ms que pertenecen a ese grupo.

He anidado el AVG() dentro de round(..., 2) para redondear el resultado a 2 decimales y obtener un informe limpio.

GROUP BY endpoint: Al igual que antes, agrupo todas las filas que comparten el mismo endpoint para que la función AVG() pueda operar sobre cada grupo.

ORDER BY avg_response_time_ms DESC: Esta es la parte más importante para el análisis. En lugar de ordenar por popularidad (conteo), he ordenado por el tiempo de respuesta promedio de forma descendente. Esto sitúa automáticamente los endpoints más lentos (los más problemáticos) en la parte superior del informe.

LIMIT 10: Limito el resultado al "Top 10" de los peores endpoints.

El resultado de esta consulta nos permite identificar qué partes de la aplicación necesitan optimización urgentemente.

# *Apartado J*

Filtra las peticiones con un tiempo de respuesta superior a la media.

### Consulta en Python

In [32]:
# --- EJERCICIO 2 - Apartado J ---

print("Filtrando peticiones con tiempo de respuesta superior a la media...")

try:
    # 1. Primero, necesitamos saber cuál es la media global
    avg_query = "SELECT AVG(duration_ms) FROM web_access_logs"
    avg_result = client.query(avg_query)

    # El resultado es [(2545.123...)]
    media_global = avg_result.result_rows[0][0]

    print(f"La media global de respuesta es: {media_global:.2f} ms")

    # 2. Ahora, contamos cuántas peticiones están POR ENCIMA de esa media
    # Usamos la variable 'media_global' que acabamos de calcular
    count_query = f"""
    SELECT 
        count()
    FROM 
        web_access_logs 
    WHERE 
        duration_ms > {media_global}
    """
    count_result = client.query(count_query)
    total_lentas = count_result.result_rows[0][0]

    print(f"Se encontraron {total_lentas} peticiones (del total de 871) por encima de la media.")

    # 3. Finalmente, mostramos una muestra del "Top 10" de las más lentas
    sample_query = f"""
    SELECT 
        endpoint,
        ip_address,
        duration_ms
    FROM 
        web_access_logs
    WHERE
        duration_ms > {media_global}
    ORDER BY 
        duration_ms DESC
    LIMIT 10
    """
    sample_result = client.query(sample_query)

    print("-------------------------------------------------")
    print("¡ÉXITO! Top 10 peticiones más lentas (por encima de la media):")
    print("-------------------------------------------------")

    if sample_result.row_count > 0:
        print(f"{'Endpoint':<30} | {'IP Address':<18} | {'Respuesta (ms)':<15}")
        print("-" * 67)
        # row[0]=endpoint, row[1]=ip, row[2]=duration
        for row in sample_result.result_rows:
            print(f"{row[0]:<30} | {row[1]:<18} | {row[2]:<15}")
    else:
        print("No se encontraron registros.")

    print("-------------------------------------------------")

except Exception as e:
    print(f"⛔ ERROR al filtrar por media: {e}")

Filtrando peticiones con tiempo de respuesta superior a la media...
La media global de respuesta es: 2533.40 ms
Se encontraron 439 peticiones (del total de 871) por encima de la media.
-------------------------------------------------
¡ÉXITO! Top 10 peticiones más lentas (por encima de la media):
-------------------------------------------------
Endpoint                       | IP Address         | Respuesta (ms) 
-------------------------------------------------------------------
/usr                           | 82.195.127.28      | 4999           
/usr/login                     | 160.103.77.2       | 4989           
/usr/login                     | 101.46.210.60      | 4987           
/usr                           | 217.156.106.254    | 4979           
/usr                           | 204.222.168.169    | 4973           
/usr/login                     | 168.20.208.75      | 4973           
/usr/admin/developer           | 70.218.223.153     | 4971           
/usr                    

### Explicación

Este apartado requiere filtrar la tabla basándonos en un valor agregado, lo cual es una operación analítica clásica. No podemos simplemente filtrar por un número fijo (ej. > 3000), sino que debemos hacerlo con respecto a un valor dinámico: la media de todos los datos.

Mi implementación se ha dividido en tres fases para un análisis claro:

Cálculo de la media (Baseline): Primero, he lanzado una consulta SELECT AVG(duration_ms) FROM ... para obtener un único número (media_global). Este número es nuestra "línea de base" o el umbral de rendimiento que separa una petición "normal" de una "lenta".

Conteo de peticiones lentas: Con la media ya calculada en Python, he lanzado una segunda consulta SELECT count() ... WHERE duration_ms > {media_global}. Esto nos da una idea de la magnitud del problema: casi la mitad de las peticiones (434 de 871) son más lentas que la media, lo cual podría sugerir una distribución de datos con una "cola larga" de peticiones muy lentas.

Muestra de las peores peticiones: Simplemente filtrar no es suficiente para un buen análisis, ya que nos devolvería 434 filas. Para hacerlo accionable, he lanzado una tercera consulta que filtra (WHERE duration_ms > {media_global}), pero además ordena (ORDER BY duration_ms DESC) y limita (LIMIT 10).

Nota técnica (Subqueries): La alternativa en SQL puro habría sido usar una subquery en la cláusula WHERE (ej. ...WHERE duration_ms > (SELECT AVG(duration_ms) FROM ...)). He optado por calcular la media en un paso separado en Python (media_global) y luego inyectar ese valor en las consultas siguientes. Esto hace el código más legible y me da la media explícitamente para poder imprimirla en el informe.

# *Apartado K*

Realiza una vista materializada (llamada “ip_request_count_mv”) que nos permita llevar un registro de las direcciones IP y el número de veces que aparecen.

### Consulta en Python

In [36]:
# --- EJERCICIO 2 - Apartado K ---

print("Creando la Vista Materializada 'ip_request_count_mv'...")

try:
    # 1. Limpiamos por si existían de una prueba anterior
    client.command("DROP VIEW IF EXISTS ip_request_count_mv")
    client.command("DROP TABLE IF EXISTS ip_request_count_table")
    print("Vistas y tablas anteriores eliminadas (si existían).")

    # 2. Creamos la tabla destino (la tabla "resumen")
    # Esta tabla usará un motor especial para agregar datos
    client.command("""
    CREATE TABLE ip_request_count_table (
        ip_address String,
        total_requests UInt64
    ) ENGINE = SummingMergeTree()
    ORDER BY ip_address
    """)
    print("Tabla 'ip_request_count_table' (destino) creada.")

    # 3. Creamos la Vista Materializada
    # Esta vista se "engancha" a 'web_access_logs'
    # Cada vez que algo se INSERTA en 'web_access_logs',
    # esta vista coge el dato, lo procesa, y lo inserta en 'ip_request_count_table'.
    client.command("""
    CREATE MATERIALIZED VIEW ip_request_count_mv
    TO ip_request_count_table
    AS 
    SELECT 
        ip_address,
        count() AS total_requests
    FROM 
        web_access_logs
    GROUP BY 
        ip_address
    """)
    print("Vista Materializada 'ip_request_count_mv' creada.")

    # 4. (IMPORTANTE) La vista solo funciona para DATOS NUEVOS.
    # Tenemos que "rellenarla" con los 871 datos que ya teníamos.
    print("Poblando la vista con los datos existentes...")
    client.command("""
    INSERT INTO ip_request_count_table
    SELECT 
        ip_address,
        count() AS total_requests
    FROM 
        web_access_logs
    GROUP BY 
        ip_address
    """)

    print("---------------------------------")
    print("¡ÉXITO! Vista Materializada creada y poblada.")
    print("---------------------------------")

    # 5. Hacemos una prueba para ver si funciona
    print("Prueba: Consultando la tabla 'ip_request_count_table' (Top 5 IPs):")
    result = client.query("SELECT * FROM ip_request_count_table ORDER BY total_requests DESC LIMIT 5")

    print(f"{'IP Address':<18} | {'Total Peticiones':<15}")
    print("-" * 37)
    for row in result.result_rows:
        print(f"{row[0]:<18} | {row[1]:<15}")

except Exception as e:
    print(f"⛔ ERROR al crear la Vista Materializada: {e}")

Creando la Vista Materializada 'ip_request_count_mv'...
Vistas y tablas anteriores eliminadas (si existían).
Tabla 'ip_request_count_table' (destino) creada.
Vista Materializada 'ip_request_count_mv' creada.
Poblando la vista con los datos existentes...
---------------------------------
¡ÉXITO! Vista Materializada creada y poblada.
---------------------------------
Prueba: Consultando la tabla 'ip_request_count_table' (Top 5 IPs):
IP Address         | Total Peticiones
-------------------------------------
204.222.168.169    | 50             
32.180.237.129     | 1              
33.32.243.166      | 1              
32.31.12.193       | 1              
32.224.240.39      | 1              


### Explicación

Este es el concepto más avanzado de la práctica. Una Vista Materializada (MV) en ClickHouse es un trigger (un disparador) que transforma y almacena datos de forma proactiva.

Diferencia clave con una VIEW normal:

Una VIEW normal es solo un "alias" o una consulta guardada. Cada vez que la consultas, la base de datos tiene que hacer todo el trabajo (escanear la tabla web_access_logs y contar 871 filas) en ese momento.

Una MATERIALIZED VIEW hace el trabajo por adelantado. Mantiene una tabla separada con los resultados ya calculados. Cuando la consultas, solo tiene que leer esa tabla-resumen (que es mucho más pequeña y rápida).

Mi implementación en detalle:

DROP...: Primero, me aseguro de limpiar cualquier intento anterior para que el script sea re-ejecutable.

CREATE TABLE ip_request_count_table ... ENGINE = SummingMergeTree(): He creado una tabla "destino" para la MV. El motor SummingMergeTree() es clave: está optimizado para agregar datos. Si insertamos ('1.1.1.1', 5) y luego ('1.1.1.1', 10), ClickHouse los "aplastará" automáticamente en una sola fila ('1.1.1.1', 15).

CREATE MATERIALIZED VIEW ... TO ip_request_count_table ... AS SELECT ...: Esta es la creación de la MV.

TO ip_request_count_table: Le dice a la MV dónde debe guardar los resultados.

AS SELECT ...: Esta es la "receta" de transformación. Le dice a la MV: "Cuando veas un nuevo lote de datos (INSERT) en web_access_logs, ejecuta esta consulta GROUP BY sobre ese lote, y guarda el resultado en ip_request_count_table".

INSERT INTO ip_request_count_table ...: Este es el paso crucial. La MV solo funciona para datos nuevos que se inserten después de su creación. Los 871 registros que ya estaban en web_access_logs no la activan. Por lo tanto, he ejecutado la consulta SELECT ... GROUP BY manualmente una vez para "poblar" o "rellenar" la tabla ip_request_count_table con los datos históricos.

A partir de este momento, si hiciéramos nuevas inserciones (como en la simulación en tiempo real), la tabla ip_request_count_table se actualizaría sola, y consultarla sería instantáneo, sin importar si web_access_logs crece a 800 mil o 800 millones de filas.

In [37]:
# --- VERIFICACIÓN FINAL ---

print("Verificando la Vista Materializada vs. la Tabla Original...")
print("(Si los números coinciden, la MV funciona perfectamente)")

# 1. Obtenemos los datos de la VISTA (rápido)
query_vista = "SELECT ip_address, total_requests FROM ip_request_count_table ORDER BY ip_address"
vista_result = client.query(query_vista).result_rows

# 2. Obtenemos los datos de la TABLA ORIGINAL (lento)
query_original = """
SELECT 
    ip_address,
    count() AS total_requests
FROM 
    web_access_logs
GROUP BY 
    ip_address
ORDER BY
    ip_address
"""
original_result = client.query(query_original).result_rows

# 3. Comparamos
if vista_result == original_result:
    print("\n---------------------------------")
    print("¡VERIFICACIÓN CORRECTA! ✅")
    print(f"La tabla resumen 'ip_request_count_table' tiene {len(vista_result)} IPs únicas.")
    print("Los conteos coinciden 1 a 1 con un recálculo manual de la tabla 'web_access_logs'.")
    print("La Vista Materializada funciona.")
    print("---------------------------------")
else:
    print("\n⛔ ¡ERROR DE VERIFICACIÓN! Los datos no coinciden.")

Verificando la Vista Materializada vs. la Tabla Original...
(Si los números coinciden, la MV funciona perfectamente)

---------------------------------
¡VERIFICACIÓN CORRECTA! ✅
La tabla resumen 'ip_request_count_table' tiene 822 IPs únicas.
Los conteos coinciden 1 a 1 con un recálculo manual de la tabla 'web_access_logs'.
La Vista Materializada funciona.
---------------------------------
